<a href="https://colab.research.google.com/github/alfonso-roldan/Neutrophils_TFG/blob/main/TFG_version_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import erf
import scipy as sc
import pymc as pm
import xarray as xr
from scipy.special import erf as erff
import arviz as az

#Importar Datos

##Sangre

In [ ]:
t=np.linspace(0,15,10**4) #tiempo
T=np.array([1,2,3,4,6,8,12,15]) #datos temporales
data_blood=np.zeros(8)
data_blood_n=np.zeros(8)
error_blood=np.zeros(8)

day1=[0.51,0.65,0.73,0.30,0.63]
day2=[28.6, 5.82, 18.0]
day3=[51.4, 53.2, 55.2, 53.0]
day4=[44, 44.6, 46.9, 48.3]
day6=[8.92, 4.52, 12]
day8=[0.25, 1.1, 0.31, 0.91, 0.49]
day12=[0.16, 0.12, 0.22, 0.26]
day15=[0.074, 0.17, 0.058, 0.42]
days=[day1,day2,day3,day4,day6,day8,day12,day15] #datos

#vector con valor medio por día
for i in range(len(days)):
    data_blood[i]=np.mean(days[i])

#vector con desv. típica por día
for i in range(len(days)):
    error_blood[i]=np.sqrt(np.var(days[i]))

#normalizar datos entre [0,100]
max_data=max(data_blood)
min_data=min(data_blood)
for i in range(len(days)):
    data_blood_n[i]=(data_blood[i]-min_data)*100/(max_data-min_data)

##Médula Osea

In [ ]:
t=np.linspace(0,15,10**4) #tiempo
T=np.array([1,2,3,4,6,8,12,15]) #datos temporales
data_bm=np.zeros(8)
data_bm_n=np.zeros(8)
error_bm=np.zeros(8)

day1=[29.2, 33.3, 29.3, 29.4, 32.8]
day2=[58.7, 52.0, 57.4]
day3=[39.5, 33.3, 32, 37.1]
day4=[20.6, 22.3, 20.2, 13.6]
day6=[1.26, 1.33, 2.63]
day8=[0.073, 1.91, 0.39, 0.44, 0.92]
day12=[0.14, 0.2, 0.23, 0.14]
day15=[0.29, 0.36, 0.25, 0.22]
days=[day1,day2,day3,day4,day6,day8,day12,day15] #datos


#vector con valor medio por día
for i in range(len(days)):
    data_bm[i]=np.mean(days[i])

#vector con desv. típica por día
for i in range(len(days)):
    error_bm[i]=np.sqrt(np.var(days[i]))

#normalizar datos entre [0,100]
max_data=max(data_bm)
min_data=min(data_bm)
for i in range(len(days)):
    data_bm_n[i]=(data_bm[i]-min_data)*100/(max_data-min_data)

##Bazo

In [ ]:
t=np.linspace(0,15,10**4) #tiempo
T=np.array([1,2,3,4,6,8,12,15]) #datos temporales
data_spleen=np.zeros(8)
data_spleen_n=np.zeros(8)
error_spleen=np.zeros(8)

day1=[15.9, 13.0, 8.15, 14.8, 13.5]
day2=[29.3, 10.2, 19.6]
day3=[30.4, 25.2, 35.7, 31.0]
day4=[6.54, 15.6, 21.9, 16.0]
day6=[1.62, 1.6, 3.05]
day8=[0.046, 0.22, 0.23, 0.29, 0.15]
day12=[0.046, 0.049, 0.04, 0.066]
day15=[0.031, 0.035, 0.029, 0.023]
days=[day1,day2,day3,day4,day6,day8,day12,day15] #datos


#vector con valor medio por día
for i in range(len(days)):
    data_spleen[i]=np.mean(days[i])

#vector con desv. típica por día
for i in range(len(days)):
    error_spleen[i]=np.sqrt(np.var(days[i]))

#normalizar datos entre [0,100]
max_data=max(data_spleen)
min_data=min(data_spleen)
for i in range(len(days)):
    data_spleen_n[i]=(data_spleen[i]-min_data)*100/(max_data-min_data)

##Tumor

In [ ]:
t=np.linspace(0,15,10**4) #tiempo
T=np.array([1,2,3,4,6,8,12,15]) #datos temporales
data_tumor=np.zeros(8)
data_tumor_n=np.zeros(8)
error_tumor=np.zeros(8)

day1=[1.64, 1.5, 3.73, 2.15, 0.14]
day2=[4.85, 0.92, 1.82]
day3=[15.1, 19.6, 21.1, 23.1]
day4=[38.1, 37.3, 39.7, 45.0]
day6=[19.1, 18.2, 25.4]
day8=[3.35, 9.56, 5.66, 5.64, 4.87]
day12=[2.95, 1.35, 1.69, 1.38]
day15=[1.64, 0.9, 1.5, 1.36]
days=[day1,day2,day3,day4,day6,day8,day12,day15] #datos

#vector con valor medio por día
for i in range(len(days)):
    data_tumor[i]=np.mean(days[i])

#vector con desv. típica por día
for i in range(len(days)):
    error_tumor[i]=np.sqrt(np.var(days[i]))

#normalizar datos entre [0,100]
max_data=max(data_tumor)
min_data=min(data_tumor)
for i in range(len(days)):
    data_tumor_n[i]=(data_tumor[i]-min_data)*100/(max_data-min_data)

#Método ABC

In [ ]:
#modelo del problema
def evo(t, n0, tau, td,sig):
    return n0 * np.exp((-1)*t/tau)*(erff((sig**2+td*tau)/(np.sqrt(2)*sig*tau)) \
           -erff((sig**2+(td+(-1)*t)*tau)/(np.sqrt(2)*sig*tau)))

def test_model(rng,n0,tau,td,sig,size=None):
    return evo(T,n0,tau,td,sig)

##Sangre

In [ ]:
with pm.Model() as model_blood_n:
    n0 = pm.Uniform('n0',lower=0,upper=10000)
    tau = pm.Uniform('tau',lower=0.05,upper=5) #tau no puede valer 0
    td = pm.Uniform('td',lower=0,upper=5)
    sig = pm.Uniform('sig',lower=0.05,upper=5) #sig no puede valer 0


    sim = pm.Simulator("sim", test_model,params=(n0, tau, td,sig), \
                       epsilon=1.5, observed=data_blood_n)
    #iniciar muestreo
    idata_blood_n = pm.sample_smc(chains=4)

In [ ]:
#representar gráfico de traza y estadístico \hat{R}
az.plot_trace(idata_blood_n)
plt.tight_layout()
plt.show()
az.rhat(idata_blood_n)

In [ ]:
#representar distribuciones a posteriori
az.plot_posterior(idata_blood_n,hdi_prob=0.95)
plt.show()

In [ ]:
#representar datos sintéticos
post = idata_blood_n.posterior
t = xr.DataArray(np.linspace(0, 15, 1000), dims=["plot_dim"])
tarray = np.linspace(0,15,1000)
evo_pp = post["n0"]*np.exp((-1)*t/post["tau"])*(erff((post["sig"]*post['sig']+ \
       post["td"]*post["tau"])/(np.sqrt(2)*post["sig"]*post["tau"]))- \
       erff((post["sig"]*post['sig']+(post["td"]+(-1)*t)*post["tau"])/ \
        (np.sqrt(2)*post["sig"]*post["tau"])))
half_stimate = []
full_stimate = []
norm_max = float(np.max(evo_pp.mean(("chain", "draw"))))
norm_min = float(np.min(evo_pp.mean(("chain", "draw"))))
max_time = int(np.where((evo_pp.mean(("chain", "draw"))-norm_min)*100/ \
           (norm_max-norm_min) == 100)[0])

_, ax = plt.subplots(figsize=(14, 6))
posterior = idata_blood_n.posterior.stack(samples=("draw", "chain"))

for i in np.random.randint(0, len(posterior["n0"]) , 100):
    sim = evo(t,posterior["n0"][i].to_numpy(), posterior["tau"][i].to_numpy(),\
            posterior["td"][i].to_numpy(),posterior["sig"][i].to_numpy())
    sim = (sim-norm_min)*100/(norm_max-norm_min)
    for ind,time in enumerate(tarray[max_time:]):

      if sim[ind+max_time]<50:
        temp = ind+max_time
        half_stimate.append((time-tarray[max_time])*24)
        break
    for ind,time in enumerate(tarray[temp:]):
      if sim[ind+temp]<5:
        full_stimate.append((time-tarray[max_time])*24)
        break
    ax.plot(t,sim, alpha=0.1, c="C0")

data_norm = (data_blood_n-norm_min)*100/(norm_max-norm_min)
ax.plot(T, data_norm, "o", label="Datos", c="r", mec="k")
ax.errorbar(T,data_norm,yerr=error_blood,fmt='none',c='r')
ax.plot(t, (evo_pp.mean(("chain", "draw"))-norm_min)*100/(norm_max-norm_min), \
        label="Media", color="khaki", alpha=0.9,linewidth=2)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Población de Neutrófilos",size=20)
ax.set_title('Sangre',size=20)
plt.xticks(size=15)
ax.yaxis.grid(True)
plt.yticks(size=15)
ax.legend(fontsize=20);

In [ ]:
#representar KDE para semivida y esperanza de vida
kernel1 = sc.stats.gaussian_kde(half_stimate)
kernel2 = sc.stats.gaussian_kde(full_stimate)
interval1 = np.linspace(np.min(half_stimate), np.max(half_stimate),100)
y1=kernel1.pdf(interval1)
interval2 = np.linspace(np.min(full_stimate), np.max(full_stimate),100)
y2=kernel2.pdf(interval2)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval1,y1,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Densidad",size=20)
ax.legend(fontsize=20)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval2,y2,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=16)
ax.legend(fontsize=16)

##Médula Ósea

In [ ]:
with pm.Model() as model_bm_n:
    n0 = pm.Uniform('n0',lower=0,upper=10000)
    tau = pm.Uniform('tau',lower=0.05,upper=5) #tau no puede valer 0
    td = pm.Uniform('td',lower=0,upper=5)
    sig = pm.Uniform('sig',lower=0.05,upper=5) #sig no puede valer 0


    sim = pm.Simulator("sim", test_model, params=(n0, tau, td,sig), \
                       epsilon=1.5, observed=data_bm_n)

    #iniciar muestreo
    idata_bm_n = pm.sample_smc(chains=4)

In [ ]:
#representar gráfico de traza y estadístico \hat{R}
az.plot_trace(idata_bm_n)
plt.tight_layout()
plt.show()
az.rhat(idata_bm_n)

In [ ]:
#representar distribución a posteriori
az.plot_posterior(idata_bm_n,hdi_prob=0.95)
plt.show()

In [ ]:
# representar datos sintéticos
post = idata_bm_n.posterior
t = xr.DataArray(np.linspace(0, 15, 1000), dims=["plot_dim"])
tarray = np.linspace(0,15,1000)
evo_pp = post["n0"]*np.exp((-1)*t/post["tau"])*(erff((post["sig"]*post['sig']+ \
       post["td"]*post["tau"])/(np.sqrt(2)*post["sig"]*post["tau"]))- \
       erff((post["sig"]*post['sig']+(post["td"]+(-1)*t)*post["tau"])/ \
        (np.sqrt(2)*post["sig"]*post["tau"])))
half_stimate = []
full_stimate = []
norm_max = float(np.max(evo_pp.mean(("chain", "draw"))))
norm_min = float(np.min(evo_pp.mean(("chain", "draw"))))
max_time = int(np.where((evo_pp.mean(("chain", "draw"))-norm_min)*100/ \
           (norm_max-norm_min) == 100)[0])

_, ax = plt.subplots(figsize=(14, 6))
posterior = idata_bm_n.posterior.stack(samples=("draw", "chain"))

for i in np.random.randint(0, len(posterior["n0"]) , 100):
    sim = evo(t,posterior["n0"][i].to_numpy(), posterior["tau"][i].to_numpy(), \
            posterior["td"][i].to_numpy(),posterior["sig"][i].to_numpy())
    sim = (sim-norm_min)*100/(norm_max-norm_min)
    for ind,time in enumerate(tarray[max_time:]):
      if sim[ind+max_time]<50:
        temp = ind+max_time
        half_stimate.append((time-tarray[max_time])*24)
        break
    for ind,time in enumerate(tarray[temp:]):
      if sim[ind+temp]<5:
        full_stimate.append((time-tarray[max_time])*24)
        break
    ax.plot(t,sim, alpha=0.1, c="C0")

data_norm=(data_bm_n-norm_min)*100/(norm_max-norm_min)
ax.plot(T, data_norm, "o", label="Datos", c="r", mec="k")
ax.errorbar(T,data_norm,yerr=error_bm,fmt='none',c='r')
ax.plot(t, (evo_pp.mean(("chain", "draw"))-norm_min)*100/(norm_max-norm_min), \
        label="Media", color="khaki", alpha=0.9,linewidth=2)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Población de Neutrófilos",size=20)
ax.set_title('Médula Ósea',size=20)
plt.xticks(size=15)
ax.yaxis.grid(True)
plt.yticks(size=15)
ax.legend(fontsize=20);

In [ ]:
#representar KDE para semivida y esperanza de vida
kernel1 = sc.stats.gaussian_kde(half_stimate)
kernel2 = sc.stats.gaussian_kde(full_stimate)
interval1 = np.linspace(np.min(half_stimate), np.max(half_stimate),100)
y1=kernel1.pdf(interval1)
interval2 = np.linspace(np.min(full_stimate), np.max(full_stimate),100)
y2=kernel2.pdf(interval2)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval1,y1,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Densidad",size=20)
ax.legend(fontsize=16,loc='upper right')

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval2,y2,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=16)
ax.legend(fontsize=16)

##Bazo

In [ ]:
with pm.Model() as model_spleen_n:
    n0=pm.Uniform('n0',lower=0,upper=10000)
    tau=pm.Uniform('tau',lower=0.05,upper=5) #tau no puede valer 0
    td=pm.Uniform('td',lower=0,upper=5)
    sig=pm.Uniform('sig',lower=0.05,upper=5) #sig no puede valer 0

    sim = pm.Simulator("sim", test_model, params=(n0, tau, td,sig), \
                       epsilon=5, observed=data_spleen_n)

    #iniciar muestreo
    idata_spleen_n = pm.sample_smc(chains=4,draws=4000)


In [ ]:
#representar gráfico de traza y estadístico \hat{R}
az.plot_trace(idata_spleen_n)
plt.tight_layout()
plt.show()
az.rhat(idata_spleen_n)

In [ ]:
#representar distribución a posteriori
az.plot_posterior(idata_spleen_n,hdi_prob=0.95)
plt.show()

In [ ]:
#representar datos sintéticos
post = idata_spleen_n.posterior
t = xr.DataArray(np.linspace(0, 15, 1000), dims=["plot_dim"])
tarray = np.linspace(0,15,1000)
evo_pp = post["n0"]*np.exp((-1)*t/post["tau"])*(erff((post["sig"]*post['sig']+ \
       post["td"]*post["tau"])/(np.sqrt(2)*post["sig"]*post["tau"]))- \
       erff((post["sig"]*post['sig']+(post["td"]+(-1)*t)*post["tau"])/ \
        (np.sqrt(2)*post["sig"]*post["tau"])))
half_stimate = []
full_stimate = []
norm_max = float(np.max(evo_pp.mean(("chain", "draw"))))
norm_min = float(np.min(evo_pp.mean(("chain", "draw"))))
max_time = int(np.where((evo_pp.mean(("chain", "draw"))-norm_min)*100/ \
           (norm_max-norm_min) == 100)[0])

_, ax = plt.subplots(figsize=(14, 6))
posterior = idata_spleen_n.posterior.stack(samples=("draw", "chain"))

for i in np.random.randint(0, len(posterior["n0"]) , 100):
    sim = evo(t,posterior["n0"][i].to_numpy(), posterior["tau"][i].to_numpy(), \
              posterior["td"][i].to_numpy(),posterior["sig"][i].to_numpy())
    sim = (sim-norm_min)*100/(norm_max-norm_min)
    for ind,time in enumerate(tarray[max_time:]):
      if sim[ind+max_time]<50:
        temp = ind+max_time
        half_stimate.append((time-tarray[max_time])*24)
        break
    for ind,time in enumerate(tarray[temp:]):
      if sim[ind+temp]<5:
        full_stimate.append((time-tarray[max_time])*24)
        break
    ax.plot(t,sim, alpha=0.1, c="C0")

data_norm = (data_spleen_n-norm_min)*100/(norm_max-norm_min)
ax.plot(T, data_norm, "o", label="Datos", c="r", mec="k")
ax.errorbar(T,data_norm,yerr=error_spleen,fmt='none',c='r')
ax.plot(t, (evo_pp.mean(("chain", "draw"))-norm_min)*100/(norm_max-norm_min), \
        label="Media", color="khaki", alpha=0.9,linewidth=2)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Población de Neutrófilos",size=20)
ax.set_title('Bazo',size=20)
plt.xticks(size=15)
ax.yaxis.grid(True)
plt.yticks(size=15)
ax.legend(fontsize=20);

In [ ]:
#representar KDE para semivida y esperanza de vida
kernel1 = sc.stats.gaussian_kde(half_stimate)
kernel2 = sc.stats.gaussian_kde(full_stimate)
interval1 = np.linspace(np.min(half_stimate), np.max(half_stimate),100)
y1=kernel1.pdf(interval1)
interval2 = np.linspace(np.min(full_stimate), np.max(full_stimate),100)
y2=kernel2.pdf(interval2)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval1,y1,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Densidad",size=20)
ax.legend(fontsize=16)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval2,y2,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=16)
ax.legend(fontsize=16)

##Tumor

In [ ]:
with pm.Model() as model_tumor_n:
    n0 = pm.Uniform('n0',lower=0,upper=10000)
    tau = pm.Uniform('tau',lower=0.05,upper=5) #tau no puede valer 0
    td = pm.Uniform('td',lower=0,upper=5)
    sig = pm.Uniform('sig',lower=0.05,upper=5) #sig no puede valer 0


    sim = pm.Simulator("sim", test_model, params=(n0, tau,td,sig), \
                       epsilon=1.5, observed=data_tumor_n)

    #iniciar muestreo
    idata_tumor_n = pm.sample_smc(chains=4)

In [ ]:
#representar gráfico de traza y estadístico \hat{R}
az.plot_trace(idata_tumor_n)
plt.tight_layout()
plt.show()
az.rhat(idata_tumor_n)

In [ ]:
#representar distribución a posteriori
az.plot_posterior(idata_tumor_n,hdi_prob=0.95)
plt.show()

In [ ]:
#representar datos sintéticos
post = idata_tumor_n.posterior
t = xr.DataArray(np.linspace(0, 15, 1000), dims=["plot_dim"])
tarray = np.linspace(0,15,1000)
evo_pp = post["n0"]*np.exp((-1)*t/post["tau"])*(erff((post["sig"]*post['sig']+ \
       post["td"]*post["tau"])/(np.sqrt(2)*post["sig"]*post["tau"]))- \
       erff((post["sig"]*post['sig']+(post["td"]+(-1)*t)*post["tau"])/ \
        (np.sqrt(2)*post["sig"]*post["tau"])))
half_stimate = []
full_stimate = []
norm_max = float(np.max(evo_pp.mean(("chain", "draw"))))
norm_min = float(np.min(evo_pp.mean(("chain", "draw"))))
max_time = int(np.where((evo_pp.mean(("chain", "draw"))-norm_min)*100/ \
           (norm_max-norm_min) == 100)[0])


_, ax = plt.subplots(figsize=(14, 6))
posterior = idata_tumor_n.posterior.stack(samples=("draw", "chain"))

for i in np.random.randint(0, len(posterior["n0"]) , 100):
    sim = evo(t,posterior["n0"][i].to_numpy(), posterior["tau"][i].to_numpy(), \
            posterior["td"][i].to_numpy(),posterior["sig"][i].to_numpy())
    sim = (sim-norm_min)*100/(norm_max-norm_min)
    for ind,time in enumerate(tarray[max_time:]):
      if sim[ind+max_time]<50:
        temp = ind+max_time
        half_stimate.append((time-tarray[max_time])*24)
        break
    for ind,time in enumerate(tarray[temp:]):
      if sim[ind+temp]<5:
        full_stimate.append((time-tarray[max_time])*24)
        break
    ax.plot(t,sim, alpha=0.1, c="C0")
data_norm = (data_tumor_n-norm_min)*100/(norm_max-norm_min)
ax.plot(T, data_norm, "o", label="Datos", c="r", mec="k")
ax.errorbar(T,data_norm,yerr=error_tumor,fmt='none',c='r')
ax.plot(t, (evo_pp.mean(("chain", "draw"))-norm_min)*100/(norm_max-norm_min), \
        label="Media", color="khaki", alpha=0.9,linewidth=2)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Población de Neutrófilos",size=20)
ax.set_title('Tumor',size=20)
plt.xticks(size=15)
ax.yaxis.grid(True)
plt.yticks(size=15)
ax.legend(fontsize=20);

In [ ]:
kernel1 = sc.stats.gaussian_kde(half_stimate)
kernel2 = sc.stats.gaussian_kde(full_stimate)
interval1 = np.linspace(np.min(half_stimate), np.max(half_stimate),100)
y1=kernel1.pdf(interval1)
interval2 = np.linspace(np.min(full_stimate), np.max(full_stimate),100)
y2=kernel2.pdf(interval2)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval1,y1,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=20)
ax.set_ylabel("Densidad",size=20)
ax.legend(fontsize=16)

_, ax = plt.subplots(figsize=(14, 6))
ax.plot(interval2,y2,label='KDE')
plt.xticks(size=15)
plt.yticks(size=15)
ax.set_xlabel("Tiempo",size=16)
ax.legend(fontsize=16)

#Test U de Mann-Whitney

In [ ]:
N = 100 #número de datos sintéticos generados
t = xr.DataArray(np.linspace(1, 15, 10000), dims=["plot_dim"]) #tiempo
eps = 0.05 #tolerancia

blood_hl = np.zeros(N)
blood_ls = np.zeros(N)
bm_hl = np.zeros(N)
bm_ls = np.zeros(N)
spleen_hl = np.zeros(N)
spleen_ls = np.zeros(N)
tumor_hl = np.zeros(N)
tumor_ls = np.zeros(N)

##Sangre

In [ ]:
posterior = idata_blood_n.posterior.stack(samples=("draw", "chain"))
L = len(t)
sample = np.random.randint(0, len(posterior["n0"]), N)
for i in range(N):
        sim = evo(t,posterior["n0"][sample[i]].to_numpy(), \
                posterior["tau"][sample[i]].to_numpy(), \
                posterior["td"][sample[i]].to_numpy(), \
                posterior["sig"][sample[i]].to_numpy())
        temp = max(sim)
        temp2 = min(sim)
        ind_temp = int(sim.argmax(axis=0))
        sim_2 = (sim-temp2)*100/(temp-temp2)
        t_max = t[ind_temp]

        #calcular la semivida
        ind_hl = 0
        for j in range(ind_temp,L):
            if abs(sim_2[j]-50)<eps:
                ind_hl = j
                t_hl = t[ind_hl]
                break
        blood_hl[i] = (t_hl-t_max)*24 #pasar de días a horas

        #calcular la esperanza de vida
        ind_ls = 0
        for j in range(ind_hl,L):
            if abs(sim_2[j]-5)<eps:
                ind_ls = j
                t_ls = t[ind_ls]
                break
        blood_ls[i] = (t_ls-t_max)*24 #pasar de días a horas



##Médula Ósea

In [ ]:
posterior = idata_bm_n.posterior.stack(samples=("draw", "chain"))
L = len(t)
sample = np.random.randint(0, len(posterior["n0"]), N)
for i in range(N):
        sim=evo(t,posterior["n0"][sample[i]].to_numpy(), \
                posterior["tau"][sample[i]].to_numpy(), \
                posterior["td"][sample[i]].to_numpy(), \
                posterior["sig"][sample[i]].to_numpy())
        temp = max(sim)
        temp2 = min(sim)
        ind_temp = int(sim.argmax(axis=0))
        sim_2 = (sim-temp2)*100/(temp-temp2)
        t_max = t[ind_temp]

        #calcular la semivida
        ind_hl = 0
        for j in range(ind_temp,L):
            if abs(sim_2[j]-50)<eps:
                ind_hl = j
                t_hl = t[ind_hl]
                break
        bm_hl[i] = (t_hl-t_max)*24 #pasar de días a horas

        #calcular la esperanza de vida
        ind_ls = 0
        for j in range(ind_hl,L):
            if abs(sim_2[j]-5)<eps:
                ind_ls = j
                t_ls = t[ind_ls]
                break
        bm_ls[i] = (t_ls-t_max)*24 #pasar de días a horas

##Bazo

In [ ]:
posterior = idata_spleen_n.posterior.stack(samples=("draw", "chain"))
L=len(t)
sample = np.random.randint(0, len(posterior["n0"]), N)
for i in range(N):
        sim=evo(t,posterior["n0"][sample[i]].to_numpy(), \
                posterior["tau"][sample[i]].to_numpy(), \
                posterior["td"][sample[i]].to_numpy(), \
                posterior["sig"][sample[i]].to_numpy())
        temp = max(sim)
        temp2 = min(sim)
        ind_temp = int(sim.argmax(axis=0))
        sim_2 = (sim-temp2)*100/(temp-temp2)
        t_max = t[ind_temp]

        #calcular la semivida
        ind_hl = 0
        for j in range(ind_temp,L):
            if abs(sim_2[j]-50)<eps:
                ind_hl = j
                t_hl = t[ind_hl]
                break
        spleen_hl[i] = (t_hl-t_max)*24 #pasar de días a horas

        #calcular la esperanza de vida
        ind_ls = 0
        for j in range(ind_hl,L):
            if abs(sim_2[j]-5)<eps:
                ind_ls = j
                t_ls = t[ind_ls]
                break
        spleen_ls[i] = (t_ls-t_max)*24 #pasar de días a horas

##Tumor

In [ ]:
posterior = idata_tumor_n.posterior.stack(samples=("draw", "chain"))
L = len(t)
sample = np.random.randint(0, len(posterior["n0"]), N)
for i in range(N):
        sim = evo(t,posterior["n0"][sample[i]].to_numpy(), \
                  posterior["tau"][sample[i]].to_numpy(), \
                  posterior["td"][sample[i]].to_numpy(), \
                  posterior["sig"][sample[i]].to_numpy())
        temp = max(sim)
        temp2 = min(sim)
        ind_temp = int(sim.argmax(axis=0))
        sim_2 = (sim-temp2)*100/(temp-temp2)
        t_max = t[ind_temp]

        #calcular la semivida
        ind_hl = 0
        for j in range(ind_temp,L):
            if abs(sim_2[j]-50)<eps:
                ind_hl = j
                t_hl = t[ind_hl]
                break
        tumor_hl[i] = (t_hl-t_max)*24 #pasar de días a horas

        #calcular la esperanza de vida
        ind_ls = 0
        for j in range(ind_hl,L):
            if abs(sim_2[j]-5)<eps:
                ind_ls = j
                t_ls = t[ind_ls]
                break
        tumor_ls[i] = (t_ls-t_max)*24 #pasar de días a horas

##Test

In [ ]:
#6 test en total (4 tejidos)
#SANGRE-MO
blood_bm_hl = sc.stats.mannwhitneyu(blood_hl,bm_hl,alternative='two-sided')
blood_bm_ls = sc.stats.mannwhitneyu(blood_ls,bm_ls, alternative='two-sided')
#SANGRE-BAZO
blood_spleen_hl = sc.stats.mannwhitneyu(blood_hl,spleen_hl, \
                                        alternative='two-sided')
blood_spleen_ls = sc.stats.mannwhitneyu(blood_ls,spleen_ls, \
                                        alternative='two-sided')
#SANGRE-TUMOR
blood_tumor_hl = sc.stats.mannwhitneyu(blood_hl,tumor_hl, \
                                       alternative='two-sided')
blood_tumor_ls = sc.stats.mannwhitneyu(blood_ls,tumor_ls, \
                                       alternative='two-sided')
#MO-BAZO
bm_spleen_hl = sc.stats.mannwhitneyu(bm_hl,spleen_hl, \
                                     alternative='two-sided')
bm_spleen_ls = sc.stats.mannwhitneyu(bm_ls,spleen_ls, \
                                     alternative='two-sided')
#MO-TUMOR
bm_tumor_hl = sc.stats.mannwhitneyu(bm_hl,tumor_hl, \
                                    alternative='two-sided')
bm_tumor_ls = sc.stats.mannwhitneyu(bm_ls,tumor_ls, \
                                    alternative='two-sided')
#BAZO-TUMOR
spleen_tumor_hl = sc.stats.mannwhitneyu(spleen_hl,tumor_hl, \
                                        alternative='two-sided')
spleen_tumor_ls = sc.stats.mannwhitneyu(spleen_ls,tumor_ls, \
                                        alternative='two-sided')

In [ ]:
print('Semivida Sangre-MO =',blood_bm_hl)

print('Semivida Sangre-Bazo =',blood_spleen_hl)

print('Semivida Sangre-Tumor =',blood_tumor_hl)

print('Semivida MO-Bazo =',bm_spleen_hl)

print('Semivida MO-Tumor =',bm_tumor_hl)

print('Semivida Bazo-Tumor =',spleen_tumor_hl)

In [ ]:
print('Esperanza de vida Sangre-MO =',blood_bm_ls)

print('Esperanza de vida Sangre-Bazo =',blood_spleen_ls)

print('Esperanza de vida Sangre-Tumor =',blood_tumor_ls)

print('Esperanza de vida MO-Bazo =',bm_spleen_ls)

print('Esperanza de vida MO-Tumor =',bm_tumor_ls)

print('Esperanza de vida Bazo-Tumor =',spleen_tumor_ls)